In [1]:
!pip install lightgbm
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install catboost

In [4]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,precision_score,recall_score,confusion_matrix, accuracy_score

In [7]:
df = pd.read_csv("data/train.csv")
df.head()

,target,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,0,0,0.468142,-1.045346,0.0,0.384487,0.435121,-1.178548,0.124543,1.801544,...,-0.361507,-1.026853,0.0,1.418600,-0.929668,1.284014,0.731842,0.801786,-0.728297,-0.412095
1,0,1,-0.760983,0.515132,0.0,-1.673905,-0.393862,-1.584207,-0.439778,0.796104,...,-0.546275,-1.489542,0.0,-0.622007,-0.473156,0.780020,0.648577,0.646100,-0.789362,0.083349
2,0,2,1.658855,0.915052,0.0,-0.581082,0.477199,-0.622226,0.390642,0.753299,...,-0.485999,0.586012,0.0,0.361481,-0.364566,-1.318596,-0.385155,0.140133,0.123245,-0.670030
3,0,3,-0.638854,0.314099,0.0,0.000919,1.102342,-0.807371,0.329158,0.484305,...,0.321985,-0.075827,0.0,-1.629672,0.876864,0.411271,0.433440,0.997364,2.829590,-1.275588
4,0,4,-1.091376,0.859811,0.0,-0.505439,1.665086,-0.912464,-0.332054,0.707705,...,0.828886,0.140387,0.0,-0.624304,-2.197691,-1.479267,-0.465917,-0.014757,-0.320434,-0.511896


In [8]:
X = df.drop(["target"],axis=1)
y = df["target"]

## BaseLine модель (LightGbm)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgb_clf = lgb.LGBMClassifier()

lgb_clf.fit(X_train,y_train)

lgb_clf.predict(X_test)

y_pred = lgb_clf.predict(X_test)

y_pred_proba = lgb_clf.predict_proba(X_test)[:, 1]

print("recall:", recall_score(y_test,y_pred))
print("precision:", precision_score(y_test,y_pred))
print("roc-auc:", roc_auc_score(y_test,y_pred_proba))
print("confusion matrix:", confusion_matrix(y_test,y_pred))

[LightGBM] [Info] Number of positive: 684, number of negative: 30300
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036768 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97565
[LightGBM] [Info] Number of data points in the train set: 30984, number of used features: 419
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022076 -> initscore=-3.790945
[LightGBM] [Info] Start training from score -3.790945
recall: 0.0
precision: 0.0
roc-auc: 0.7648786730454905
confusion matrix: [[7588    0]
 [ 159    0]]


C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Модель Catboost

In [11]:
import catboost as ctb
from catboost import CatBoostClassifier

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

cat_clf = CatBoostClassifier(
    iterations=100,              
    learning_rate=0.1,             
    depth=10,                       
    class_weights=class_weights,   
    eval_metric='AUC',             
    random_seed=42,               
    verbose=5   
)

In [12]:
cat_clf.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),  
    early_stopping_rounds=50           
)

0:	test: 0.6870941	best: 0.6870941 (0)	total: 790ms	remaining: 1m 18s
5:	test: 0.7280885	best: 0.7280885 (5)	total: 3.96s	remaining: 1m 2s
10:	test: 0.7448528	best: 0.7448528 (10)	total: 7.03s	remaining: 56.9s
15:	test: 0.7611613	best: 0.7611613 (15)	total: 10.1s	remaining: 52.9s
20:	test: 0.7610021	best: 0.7611613 (15)	total: 13.1s	remaining: 49.4s
25:	test: 0.7547551	best: 0.7611613 (15)	total: 16.2s	remaining: 46s
30:	test: 0.7491314	best: 0.7611613 (15)	total: 19.2s	remaining: 42.8s
35:	test: 0.7473386	best: 0.7611613 (15)	total: 22.2s	remaining: 39.5s
40:	test: 0.7458118	best: 0.7611613 (15)	total: 25.2s	remaining: 36.3s
45:	test: 0.7468106	best: 0.7611613 (15)	total: 28.2s	remaining: 33.1s
50:	test: 0.7463589	best: 0.7611613 (15)	total: 31.3s	remaining: 30.1s
55:	test: 0.7480340	best: 0.7611613 (15)	total: 34.4s	remaining: 27s
60:	test: 0.7481533	best: 0.7611613 (15)	total: 37.4s	remaining: 23.9s
65:	test: 0.7491198	best: 0.7611613 (15)	total: 40.5s	remaining: 20.9s
Stopped by ov

In [13]:
y_pred = cat_clf.predict(X_test)

In [14]:
y_pred_proba = cat_clf.predict_proba(X_test)[:, 1]

In [15]:
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Recall:", recall)
print("Precision:", precision)
print("ROC-AUC:", roc_auc)
print("Confusion Matrix:\n", conf_matrix)

Recall: 0.4339622641509434
Precision: 0.07217573221757322
ROC-AUC: 0.7611612841195798
Confusion Matrix:
 [[6701  887]
 [  90   69]]
